In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import pdfplumber
import regex as re
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np
import nltk
import requests
from io import BytesIO
import imblearn
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet') 
import re
import contractions
from sklearn.preprocessing import LabelEncoder
from bs4 import BeautifulSoup 
pd.set_option('display.max_colwidth', None) 
import requests
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import gensim.downloader as api
from gensim.models import Word2Vec
wv = api.load('word2vec-google-news-300')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yasmi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yasmi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
# import PyPDF2
# pdfReader = PyPDF2.PdfFileReader("DCISC-24th-Annual-Report-Copy1.pdf")
# count = pdfReader.numPages
# output = []
# for i in range(count):
#     page = pdfReader.getPage(i)
#     output.append(page.extractText())

In [2]:
import PyPDF2
from PyPDF2 import PdfReader

reader = PdfReader("DCISC-24th-Annual-Report-Copy1.pdf")
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"

In [3]:
# tokenize the df based on where sentences end. 
from nltk import tokenize
text_to_tokenize = text
tokenized_text = tokenize.sent_tokenize(text_to_tokenize)

In [4]:
# make a dataframe of the tokenized sentences above and make the text in lower case
df = pd.DataFrame (tokenized_text, columns = ['text'])
df['text'] = df['text'].apply(lambda x:x.lower())
df.head()

,text
0,"developed by virtual silk®24th annual report by the diablo canyon independent safety committee, july 1, 2013 - june 30, 2014\npreface | executive summary\nvolume 1 toc | volume 2 toc | pg&e response | contact the dcisc\ndiablo canyon independent safety committee\ntwenty-fourth annual report on the safety of diablo canyon nuclear power plant\noperations\njuly 1, 2013 – june 30, 2014\npeter lam, chair\nrobert k budnitz, vice-chair\nper f. peterson, member\n24th annual report by the diablo canyon independent safety committee, july 1, 2013 - june 30, 2014\npreface | executive summary\nvolume 1 toc | volume 2 toc | pg&e response | contact the dcisc\n24th annual report , preface\nthis report covers the activities of the diablo canyon independent safety committee (dcisc)\nfor the period july 1, 2013 through june 30, 2014."
1,this is the twenty-fourth annual report of the\ndcisc.
2,the report is presented in two volumes.
3,"volume i includes a report summary and conclusions and recommendation ( executive summary ), a\nbrief introduction and history regarding the dcisc, committee activities, and documents receivedby the dcisc during the reporting period \n(section 1.0 ), dcisc public meetings ( section 2.0 ), a\nreview and evaluation of nuclear regulatory commission (nrc) assessments and issues ( section\n3.0), committee member and consultant investigation topical summaries ( section 4.0 ), dcpp\nperformance indicators monitored by the dcisc ( section 5.0 ), open items being followed by the\ncommittee ( section 6.0 ), follow-up of pacific gas and electric (pg&e) actions on previous dcisc\nrecommendations ( section 7.0 ), input to the committee by members of the public ( section 8.0 ),\nand pg&e’s response ( section 9.0 ) to recommendation in this report."
4,the conclusions and\nrecommendation also appear in bold face type throughout the main body of the report with a\ndiscussion of the subject involved.


In [5]:
def remove_dates_titles_volume(sentence):
    sentence = sentence.replace('\n'," ")
    sentence = re.sub('[^\w\s]', '', sentence)
    sentence = re.sub('{January|February|March|April|May|June|July|August|September|October|November|December}', '', sentence)
    sentence = re.sub('\d', '', sentence)
    prefixes = "(Mr|St|Mrs|Ms|Dr|Ph.D|Chair|ViceChair|Volume)"
    sentence  = re.sub(prefixes, '', sentence)
    return sentence

def remove_stop_words(txt):
    stop = stopwords.words('english') 
    word_list = txt.split() 
    clean_list = []
    clean_string = ''
    for word in word_list:
        if word not in stop: 
            clean_list.append(word)
    clean_string = ' '.join(clean_list) 
    return clean_string

def contractionfunction(s):
    expanded_words = []
    for word in s.split():
        expanded_words.append(contractions.fix(word))
    result = ' '.join(expanded_words)
    return result

def remove_non_alphabetical(txt):
    regex = re.compile('[\W_0-9]+')
    dirty_list = txt.split()
    clean_list = [regex.sub(' ', word) for word in dirty_list] 
    clean_string = ' '.join(clean_list)
    return clean_string

def leammatize_review(txt): 
    lemmatizer = WordNetLemmatizer() 
    word_list = txt.split() 
    clean_list = []
    clean_string = ''
    for word in word_list:
        new_word = lemmatizer.lemmatize(word)
        clean_list.append(new_word) 
    clean_string = ' '.join(clean_list) 
    return clean_string

In [6]:
# Applying preproccessing
df['text'] = df['text'].apply(lambda body :remove_dates_titles_volume(body))
df['text'] = df['text'].apply(lambda body :contractionfunction(body))
df['text'] = df['text'].apply(lambda body :remove_non_alphabetical(body))
df['text'] = df['text'].apply(lambda review:remove_stop_words(review))
df['text'] = df['text'].apply(lambda txt :leammatize_review(txt))

In [25]:
safety_traits = ['Personal Accountability', 'Questioning Attitue', 'Effective Safety Communication', 
                 'Leadership Safety Values and Actions','Decision Making', 'Respectful Work Environment', 
                 'Continuous Learning','Problem Identification and Resolution','Environment for Raising Concerns',
                 'Work Processes']

# seed_words = [['responsible', 'help', 'ownership', 'support', 'trained', 'qualified', 'understand', 'involvement'],
#               ['complacent', 'discrepancy', 'anomaly', 'assumption', 'question', 'uncertain', 'unknown', 'risk', 'investigate', 'unexpected', 'unclear', 'review', 'degrading', 'aging'],
#               ['communication', 'information', 'safety', 'share', 'respond', 'listen', 'concerns', 'expectations', 'clear'],
#               ['leadership', 'management', 'leaders', 'owner', 'guidance', 'decisions', 'policies', 'resources', 'staffing', 'expectations', 'oversight', 'reinforce', 'decision-making', 'mentoring', 'coaching', 'encourage', 'incentive', 'reward', 'priorities', 'plan', 'delegate', 'define', 'manage', 'support', 'resolve', 'motivate', 'address', 'resources', 'translate', 'funding', 'implementation'],
#               ['decision-making', 'thorough', 'conservative', 'systematic', 'consistent', 'process', 'leaders', 'choices', 'consequences', 'reinforce', 'accountable', 'authority', 'responsibility'],
#               ['trust', 'respect', 'encourage', 'opinions', 'bullying', 'fair', 'productive', 'concerns', 'voice', 'discussions', 'value', 'insight', 'perspectives', 'collaboration', 'conflict', 'listening'],
#               ['learn', 'training', 'self-assessments', 'improve', 'performance', 'monitor', 'experience', 'assessment', 'corrective action', 'benchmarking', 'knowledge', 'competence', 'skills', 'develop', 'understand'],
#               ['identify', 'address', 'correct', 'corrective action', 'issues', 'deviation', 'degraded conditions', 'evaluation', 'problems', 'root cause', 'investigation', 'recommendation', 'cause analysis', 'resolution', 'mitigate', 'trends'],
#               ['concerns', 'confidence', 'address', 'feedback', 'listening', 'report'],
#               ['plan', 'control', 'activities', 'production', 'schedule', 'manage', 'execute', 'risk', 'coordinate', 'margins', 'operate', 'maintain', 'maintenance', 'document', 'complete', 'procedures', 'packages', 'accurate', 'current', 'backlog', 'follow', 'review', 'instructions', 'operations', 'design', 'requirements', 'operations']]

seed_words = [['responsible', 'help', 'ownership', 'support', 'trained', 'qualified', 'understand', 'involvement'],
              ['discrepancy', 'anomaly', 'assumption', 'question', 'uncertain', 'unknown', 'risk', 'investigate', 'unexpected', 'unclear', 'review', 'aging'],
              ['communication', 'information', 'safety', 'share', 'respond', 'listen', 'clear'],
              ['leadership', 'management', 'leader', 'owner', 'guidance', 'decision', 'policy', 'resource', 'staffing', 'expectation', 'oversight', 'reinforce', 'mentor', 'coaching',  'incentive', 'priority', 'plan',  'define', 'manage', 'support', 'resolve', 'address', 'resource', 'fund', 'implement'],
              [ 'thorough', 'conservative', 'systematic', 'consistent', 'process', 'leader', 'choice', 'consequence', 'reinforce', 'authority', 'responsibility'],
              ['trust', 'respect', 'opinion',  'concern',  'discussion', 'value', 'insight', 'perspective', 'collaboration', 'conflict', 'listen'],
              ['learn', 'training', 'self', 'assessment', 'improve', 'performance', 'monitor', 'experience',  'corrective', 'action', 'benchmarking', 'knowledge', 'competence', 'skill', 'develop', 'understand'],
              ['identify', 'address', 'correct', 'corrective', 'action', 'issue', 'deviation', 'degrade', 'condition', 'evaluation', 'problem', 'root', 'cause', 'investigation', 'recommendation', 'cause', 'analysis', 'resolution', 'mitigate', 'trend'],
              ['concern', 'confidence', 'address', 'feedback', 'listen', 'report'],
              ['plan', 'control', 'activity', 'production', 'schedule', 'manage', 'risk', 'coordinate', 'margin', 'operate', 'maintain', 'maintenance', 'document', 'complete', 'procedure', 'package', 'accurate', 'current', 'backlog', 'follow', 'review', 'instruction', 'operation', 'design', 'requirement']]

In [26]:
a = pd.DataFrame(safety_traits, columns = ['Safety_Traits'])
a

,Safety_Traits
0,Personal Accountability
1,Questioning Attitue
2,Effective Safety Communication
3,Leadership Safety Values and Actions
4,Decision Making
5,Respectful Work Environment
6,Continuous Learning
7,Problem Identification and Resolution
8,Environment for Raising Concerns
9,Work Processes


In [27]:
d = pd.DataFrame(seed_words)
d['Keyword'] = d[d.columns[0:]].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
d
f = d.loc[:, ['Keyword']]
f

,Keyword
0,"responsible,help,ownership,support,trained,qualified,understand,involvement"
1,"discrepancy,anomaly,assumption,question,uncertain,unknown,risk,investigate,unexpected,unclear,review,aging"
2,"communication,information,safety,share,respond,listen,clear"
3,"leadership,management,leader,owner,guidance,decision,policy,resource,staffing,expectation,oversight,reinforce,mentor,coaching,incentive,priority,plan,define,manage,support,resolve,address,resource,fund,implement"
4,"thorough,conservative,systematic,consistent,process,leader,choice,consequence,reinforce,authority,responsibility"
5,"trust,respect,opinion,concern,discussion,value,insight,perspective,collaboration,conflict,listen"
6,"learn,training,self,assessment,improve,performance,monitor,experience,corrective,action,benchmarking,knowledge,competence,skill,develop,understand"
7,"identify,address,correct,corrective,action,issue,deviation,degrade,condition,evaluation,problem,root,cause,investigation,recommendation,cause,analysis,resolution,mitigate,trend"
8,"concern,confidence,address,feedback,listen,report"
9,"plan,control,activity,production,schedule,manage,risk,coordinate,margin,operate,maintain,maintenance,document,complete,procedure,package,accurate,current,backlog,follow,review,instruction,operation,design,requirement"


In [28]:
labels = pd.concat([a,f], axis = 1)
labels

,Safety_Traits,Keyword
0,Personal Accountability,"responsible,help,ownership,support,trained,qualified,understand,involvement"
1,Questioning Attitue,"discrepancy,anomaly,assumption,question,uncertain,unknown,risk,investigate,unexpected,unclear,review,aging"
2,Effective Safety Communication,"communication,information,safety,share,respond,listen,clear"
3,Leadership Safety Values and Actions,"leadership,management,leader,owner,guidance,decision,policy,resource,staffing,expectation,oversight,reinforce,mentor,coaching,incentive,priority,plan,define,manage,support,resolve,address,resource,fund,implement"
4,Decision Making,"thorough,conservative,systematic,consistent,process,leader,choice,consequence,reinforce,authority,responsibility"
5,Respectful Work Environment,"trust,respect,opinion,concern,discussion,value,insight,perspective,collaboration,conflict,listen"
6,Continuous Learning,"learn,training,self,assessment,improve,performance,monitor,experience,corrective,action,benchmarking,knowledge,competence,skill,develop,understand"
7,Problem Identification and Resolution,"identify,address,correct,corrective,action,issue,deviation,degrade,condition,evaluation,problem,root,cause,investigation,recommendation,cause,analysis,resolution,mitigate,trend"
8,Environment for Raising Concerns,"concern,confidence,address,feedback,listen,report"
9,Work Processes,"plan,control,activity,production,schedule,manage,risk,coordinate,margin,operate,maintain,maintenance,document,complete,procedure,package,accurate,current,backlog,follow,review,instruction,operation,design,requirement"


In [29]:
# split keywords by separator and save them as array
labels['Keyword'] = labels['Keyword'].apply(lambda x: x.split(','))

# convert description keywords to lowercase
labels['Keyword'] = labels['Keyword'].apply(lambda description_keywords: [keyword.lower() for keyword in description_keywords])

# get number of keywords for each class
labels['number_of_keywords'] = labels['Keyword'].apply(lambda row: len(row))

labels

,Safety_Traits,Keyword,number_of_keywords
0,Personal Accountability,"[responsible, help, ownership, support, trained, qualified, understand, involvement]",8
1,Questioning Attitue,"[discrepancy, anomaly, assumption, question, uncertain, unknown, risk, investigate, unexpected, unclear, review, aging]",12
2,Effective Safety Communication,"[communication, information, safety, share, respond, listen, clear]",7
3,Leadership Safety Values and Actions,"[leadership, management, leader, owner, guidance, decision, policy, resource, staffing, expectation, oversight, reinforce, mentor, coaching, incentive, priority, plan, define, manage, support, resolve, address, resource, fund, implement]",25
4,Decision Making,"[thorough, conservative, systematic, consistent, process, leader, choice, consequence, reinforce, authority, responsibility]",11
5,Respectful Work Environment,"[trust, respect, opinion, concern, discussion, value, insight, perspective, collaboration, conflict, listen]",11
6,Continuous Learning,"[learn, training, self, assessment, improve, performance, monitor, experience, corrective, action, benchmarking, knowledge, competence, skill, develop, understand]",16
7,Problem Identification and Resolution,"[identify, address, correct, corrective, action, issue, deviation, degrade, condition, evaluation, problem, root, cause, investigation, recommendation, cause, analysis, resolution, mitigate, trend]",20
8,Environment for Raising Concerns,"[concern, confidence, address, feedback, listen, report]",6
9,Work Processes,"[plan, control, activity, production, schedule, manage, risk, coordinate, margin, operate, maintain, maintenance, document, complete, procedure, package, accurate, current, backlog, follow, review, instruction, operation, design, requirement]",25


In [12]:
df

,text
0,developed virtual silkth annual report diablo canyon independent safety committee july june preface executive summary volume toc volume toc pge response contact dcisc diablo canyon independent safety committee twentyfourth annual report safety diablo canyon nuclear power plant operation july june peter lam chair robert k budnitz vicechair per f peterson member th annual report diablo canyon independent safety committee july june preface executive summary volume toc volume toc pge response contact dcisc th annual report preface report cover activity diablo canyon independent safety committee dcisc period july june
1,twentyfourth annual report dcisc
2,report presented two volume
3,volume includes report summary conclusion recommendation executive summary brief introduction history regarding dcisc committee activity document receivedby dcisc reporting period section dcisc public meeting section review evaluation nuclear regulatory commission nrc assessment issue section committee member consultant investigation topical summary section dcpp performance indicator monitored dcisc section open item followed committee section followup pacific gas electric pge action previous dcisc recommendation section input committee member public section pges response section recommendation report
4,conclusion recommendation also appear bold face type throughout main body report discussion subject involved
...,...
10683,mr geesman remarked february public meeting reference need allow pge review december fact finding report based upon pges assertion contained confidentialinformation
10684,mr geesman stated wanted raise concern action created problemunder california bagley keene open meeting act
10685,dr gene nelson recognized following consideration item open item list
10686,mr nelson stated remark concerned effect coronal mass ejection hisunderstanding event would affect power transmission line located high latitudesmostly eastern portion u western portion would significantlyvulnerable event


In [13]:
from gensim.models.doc2vec import TaggedDocument
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import TaggedDocument
from gensim.parsing.preprocessing import strip_tags

def tokenize(doc):
    return simple_preprocess(strip_tags(doc), deacc=True, min_len=2)


df['tagged_docs'] = df.apply(lambda row: TaggedDocument(tokenize(row['text']), [str(row.name)]), axis=1)

In [14]:
from gensim.models.doc2vec import TaggedDocument
df['tagged_docs'] = df.apply(lambda row: TaggedDocument(tokenize(row['text']), [str(row.name)]), axis=1)

In [16]:
# add doc_key column
df['doc_key'] = df.index.astype(str)

In [17]:
df.head()

,text,tagged_docs,doc_key
0,developed virtual silkth annual report diablo canyon independent safety committee july june preface executive summary volume toc volume toc pge response contact dcisc diablo canyon independent safety committee twentyfourth annual report safety diablo canyon nuclear power plant operation july june peter lam chair robert k budnitz vicechair per f peterson member th annual report diablo canyon independent safety committee july june preface executive summary volume toc volume toc pge response contact dcisc th annual report preface report cover activity diablo canyon independent safety committee dcisc period july june,"([developed, virtual, silkth, annual, report, diablo, canyon, independent, safety, committee, july, june, preface, executive, summary, volume, toc, volume, toc, pge, response, contact, dcisc, diablo, canyon, independent, safety, committee, twentyfourth, annual, report, safety, diablo, canyon, nuclear, power, plant, operation, july, june, peter, lam, chair, robert, budnitz, vicechair, per, peterson, member, th, annual, report, diablo, canyon, independent, safety, committee, july, june, preface, executive, summary, volume, toc, volume, toc, pge, response, contact, dcisc, th, annual, report, preface, report, cover, activity, diablo, canyon, independent, safety, committee, dcisc, period, july, june], [0])",0
1,twentyfourth annual report dcisc,"([twentyfourth, annual, report, dcisc], [1])",1
2,report presented two volume,"([report, presented, two, volume], [2])",2
3,volume includes report summary conclusion recommendation executive summary brief introduction history regarding dcisc committee activity document receivedby dcisc reporting period section dcisc public meeting section review evaluation nuclear regulatory commission nrc assessment issue section committee member consultant investigation topical summary section dcpp performance indicator monitored dcisc section open item followed committee section followup pacific gas electric pge action previous dcisc recommendation section input committee member public section pges response section recommendation report,"([volume, includes, report, summary, conclusion, recommendation, executive, summary, brief, introduction, history, regarding, dcisc, committee, activity, document, receivedby, dcisc, reporting, period, section, dcisc, public, meeting, section, review, evaluation, nuclear, regulatory, commission, nrc, assessment, issue, section, committee, member, consultant, investigation, topical, summary, section, dcpp, performance, indicator, monitored, dcisc, section, open, item, followed, committee, section, followup, pacific, gas, electric, pge, action, previous, dcisc, recommendation, section, input, committee, member, public, section, pges, response, section, recommendation, report], [3])",3
4,conclusion recommendation also appear bold face type throughout main body report discussion subject involved,"([conclusion, recommendation, also, appear, bold, face, type, throughout, main, body, report, discussion, subject, involved], [4])",4


In [27]:
print(df.shape)
print(labels.shape)

(10688, 3)
(10, 3)


In [30]:
labels = labels.rename(columns={"Safety_Traits": "class_name", "Keyword": "keywords"})
labels

,class_name,keywords,number_of_keywords
0,Personal Accountability,"[responsible, help, ownership, support, trained, qualified, understand, involvement]",8
1,Questioning Attitue,"[discrepancy, anomaly, assumption, question, uncertain, unknown, risk, investigate, unexpected, unclear, review, aging]",12
2,Effective Safety Communication,"[communication, information, safety, share, respond, listen, clear]",7
3,Leadership Safety Values and Actions,"[leadership, management, leader, owner, guidance, decision, policy, resource, staffing, expectation, oversight, reinforce, mentor, coaching, incentive, priority, plan, define, manage, support, resolve, address, resource, fund, implement]",25
4,Decision Making,"[thorough, conservative, systematic, consistent, process, leader, choice, consequence, reinforce, authority, responsibility]",11
5,Respectful Work Environment,"[trust, respect, opinion, concern, discussion, value, insight, perspective, collaboration, conflict, listen]",11
6,Continuous Learning,"[learn, training, self, assessment, improve, performance, monitor, experience, corrective, action, benchmarking, knowledge, competence, skill, develop, understand]",16
7,Problem Identification and Resolution,"[identify, address, correct, corrective, action, issue, deviation, degrade, condition, evaluation, problem, root, cause, investigation, recommendation, cause, analysis, resolution, mitigate, trend]",20
8,Environment for Raising Concerns,"[concern, confidence, address, feedback, listen, report]",6
9,Work Processes,"[plan, control, activity, production, schedule, manage, risk, coordinate, margin, operate, maintain, maintenance, document, complete, procedure, package, accurate, current, backlog, follow, review, instruction, operation, design, requirement]",25


In [31]:
labels['class_index'] = labels.index+1

In [20]:
labels.head()

,class_name,keywords,number_of_keywords,class_index
0,Personal Accountability,"[responsible, help, ownership, support, trained, qualified, understand, involvement]",8,1
1,Questioning Attitue,"[discrepancy, anomaly, assumption, question, uncertain, unknown, risk, investigate, unexpected, unclear, review, aging]",12,2
2,Effective Safety Communication,"[communication, information, safety, share, respond, listen, clear]",7,3
3,Leadership Safety Values and Actions,"[leadership, management, leader, owner, guidance, decision, policy, resource, staffing, expectation, oversight, reinforce, mentor, coaching, incentive, priority, plan, delegate, define, manage, support, resolve, address, resource, translate, fund, implement]",27,4
4,Decision Making,"[thorough, conservative, systematic, consistent, process, leader, choice, consequence, reinforce, authority, responsibility]",11,5


In [68]:
df.head()

,text,tagged_docs,doc_key
0,developed virtual silkth annual report diablo canyon independent safety committee july june preface executive summary volume toc volume toc pge response contact dcisc diablo canyon independent safety committee twentyfourth annual report safety diablo canyon nuclear power plant operation july june peter lam chair robert k budnitz vicechair per f peterson member th annual report diablo canyon independent safety committee july june preface executive summary volume toc volume toc pge response contact dcisc th annual report preface report cover activity diablo canyon independent safety committee dcisc period july june,"([developed, virtual, silkth, annual, report, diablo, canyon, independent, safety, committee, july, june, preface, executive, summary, volume, toc, volume, toc, pge, response, contact, dcisc, diablo, canyon, independent, safety, committee, twentyfourth, annual, report, safety, diablo, canyon, nuclear, power, plant, operation, july, june, peter, lam, chair, robert, budnitz, vicechair, per, peterson, member, th, annual, report, diablo, canyon, independent, safety, committee, july, june, preface, executive, summary, volume, toc, volume, toc, pge, response, contact, dcisc, th, annual, report, preface, report, cover, activity, diablo, canyon, independent, safety, committee, dcisc, period, july, june], [0])",0
1,twentyfourth annual report dcisc,"([twentyfourth, annual, report, dcisc], [1])",1
2,report presented two volume,"([report, presented, two, volume], [2])",2
3,volume includes report summary conclusion recommendation executive summary brief introduction history regarding dcisc committee activity document receivedby dcisc reporting period section dcisc public meeting section review evaluation nuclear regulatory commission nrc assessment issue section committee member consultant investigation topical summary section dcpp performance indicator monitored dcisc section open item followed committee section followup pacific gas electric pge action previous dcisc recommendation section input committee member public section pges response section recommendation report,"([volume, includes, report, summary, conclusion, recommendation, executive, summary, brief, introduction, history, regarding, dcisc, committee, activity, document, receivedby, dcisc, reporting, period, section, dcisc, public, meeting, section, review, evaluation, nuclear, regulatory, commission, nrc, assessment, issue, section, committee, member, consultant, investigation, topical, summary, section, dcpp, performance, indicator, monitored, dcisc, section, open, item, followed, committee, section, followup, pacific, gas, electric, pge, action, previous, dcisc, recommendation, section, input, committee, member, public, section, pges, response, section, recommendation, report], [3])",3
4,conclusion recommendation also appear bold face type throughout main body report discussion subject involved,"([conclusion, recommendation, also, appear, bold, face, type, throughout, main, body, report, discussion, subject, involved], [4])",4


In [ ]:
#df_agg = df.merge(labels, left_on='class', right_on='class_index', how='left').drop(['class', 'keywords'], axis=1)

In [21]:
df['class'] = df.index+1
df

,text,tagged_docs,doc_key,class
0,developed virtual silkth annual report diablo canyon independent safety committee july june preface executive summary volume toc volume toc pge response contact dcisc diablo canyon independent safety committee twentyfourth annual report safety diablo canyon nuclear power plant operation july june peter lam chair robert k budnitz vicechair per f peterson member th annual report diablo canyon independent safety committee july june preface executive summary volume toc volume toc pge response contact dcisc th annual report preface report cover activity diablo canyon independent safety committee dcisc period july june,"([developed, virtual, silkth, annual, report, diablo, canyon, independent, safety, committee, july, june, preface, executive, summary, volume, toc, volume, toc, pge, response, contact, dcisc, diablo, canyon, independent, safety, committee, twentyfourth, annual, report, safety, diablo, canyon, nuclear, power, plant, operation, july, june, peter, lam, chair, robert, budnitz, vicechair, per, peterson, member, th, annual, report, diablo, canyon, independent, safety, committee, july, june, preface, executive, summary, volume, toc, volume, toc, pge, response, contact, dcisc, th, annual, report, preface, report, cover, activity, diablo, canyon, independent, safety, committee, dcisc, period, july, june], [0])",0,1
1,twentyfourth annual report dcisc,"([twentyfourth, annual, report, dcisc], [1])",1,2
2,report presented two volume,"([report, presented, two, volume], [2])",2,3
3,volume includes report summary conclusion recommendation executive summary brief introduction history regarding dcisc committee activity document receivedby dcisc reporting period section dcisc public meeting section review evaluation nuclear regulatory commission nrc assessment issue section committee member consultant investigation topical summary section dcpp performance indicator monitored dcisc section open item followed committee section followup pacific gas electric pge action previous dcisc recommendation section input committee member public section pges response section recommendation report,"([volume, includes, report, summary, conclusion, recommendation, executive, summary, brief, introduction, history, regarding, dcisc, committee, activity, document, receivedby, dcisc, reporting, period, section, dcisc, public, meeting, section, review, evaluation, nuclear, regulatory, commission, nrc, assessment, issue, section, committee, member, consultant, investigation, topical, summary, section, dcpp, performance, indicator, monitored, dcisc, section, open, item, followed, committee, section, followup, pacific, gas, electric, pge, action, previous, dcisc, recommendation, section, input, committee, member, public, section, pges, response, section, recommendation, report], [3])",3,4
4,conclusion recommendation also appear bold face type throughout main body report discussion subject involved,"([conclusion, recommendation, also, appear, bold, face, type, throughout, main, body, report, discussion, subject, involved], [4])",4,5
...,...,...,...,...
10683,mr geesman remarked february public meeting reference need allow pge review december fact finding report based upon pges assertion contained confidentialinformation,"([mr, geesman, remarked, february, public, meeting, reference, need, allow, pge, review, december, fact, finding, report, based, upon, pges, assertion, contained], [10683])",10683,10684
10684,mr geesman stated wanted raise concern action created problemunder california bagley keene open meeting act,"([mr, geesman, stated, wanted, raise, concern, action, created, problemunder, california, bagley, keene, open, meeting, act], [10684])",10684,10685
10685,dr gene nelson recognized following consideration item open item list,"([dr, gene, nelson, recognized, following, consideration, item, open, item, list], [10685])",10685,10686
10686,mr nelson stated remark concerned effect coronal mass ejection hisunderstanding eve

In [22]:
from lbl2vec import Lbl2Vec
from gensim.models.doc2vec import TaggedDocument, Doc2Vec


In [32]:
# init model with parameters
Lbl2Vec_model = Lbl2Vec(keywords_list=list(labels['keywords']), tagged_documents=df['tagged_docs'], label_names=list(labels['class_name']), min_count = 2, similarity_threshold=0.3, min_num_docs=5, epochs=10)

In [33]:
# train model
Lbl2Vec_model.fit()

2023-03-17 15:28:47,367 - Lbl2Vec - INFO - Train document and word embeddings
2023-03-17 15:28:47,367 - Lbl2Vec - INFO - Train document and word embeddings
2023-03-17 15:29:05,215 - Lbl2Vec - INFO - Train label embeddings
2023-03-17 15:29:05,215 - Lbl2Vec - INFO - Train label embeddings


In [37]:
# predict similarity scores
model_docs_lbl_similarities = Lbl2Vec_model.predict_model_docs()

2023-03-17 15:35:12,029 - Lbl2Vec - INFO - Get document embeddings from model
2023-03-17 15:35:12,029 - Lbl2Vec - INFO - Get document embeddings from model
2023-03-17 15:35:12,061 - Lbl2Vec - INFO - Calculate document<->label similarities
2023-03-17 15:35:12,061 - Lbl2Vec - INFO - Calculate document<->label similarities


In [ ]:
y_true_train = evaluation_train['class_name']
y_pred_train = evaluation_train['most_similar_label']
print('F1 score:',f1_score(y_true_train, y_pred_train, average='micro'))

In [39]:
model_docs_lbl_similarities

,doc_key,most_similar_label,highest_similarity_score,Personal Accountability,Questioning Attitue,Effective Safety Communication,Leadership Safety Values and Actions,Decision Making,Respectful Work Environment,Continuous Learning,Problem Identification and Resolution,Environment for Raising Concerns,Work Processes
0,0,Work Processes,0.400110,0.400081,0.395845,0.400104,0.400082,0.400082,0.400107,0.400108,0.400101,0.400106,0.400110
1,1,Work Processes,0.890802,0.890789,0.888862,0.890798,0.890789,0.890788,0.890800,0.890801,0.890798,0.890800,0.890802
2,2,Work Processes,0.825830,0.825819,0.824274,0.825824,0.825819,0.825818,0.825828,0.825829,0.825827,0.825828,0.825830
3,3,Work Processes,0.578355,0.578333,0.575382,0.578349,0.578331,0.578331,0.578352,0.578352,0.578347,0.578351,0.578355
4,4,Personal Accountability,0.899478,0.899478,0.899448,0.899475,0.899477,0.899477,0.899475,0.899477,0.899477,0.899476,0.899477
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10683,10683,Questioning Attitue,0.825832,0.825720,0.825832,0.825722,0.825719,0.825719,0.825723,0.825721,0.825722,0.825721,0.825721
10684,10684,Questioning Attitue,0.904628,0.904194,0.904628,0.904192,0.904196,0.904196,0.904191,0.904190,0.904190,0.904190,0.904189
10685,10685,Respectful Work Environment,0.771538,0.771533,0.770923,0.771537,0.771532,0.771534,0.771538,0.771535,0.771536,0.771536,0.771536
10686,10686,Respectful Work Environment,0.879465,0.879464,0.878938,0.879464,0.879464,0.879463,0.879465,0.879464,0.879464,0.879464,0.879464


In [35]:
test_data = pd.read_csv('test_set.csv')
test_data

,sentence,labels,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"DCISC identified 11 Non-cited Violations, one Severity Level IV violation.",leadership safety values and actions,problem identification and resolution,personal accountability,environment for raising concerns,effective safety communication,questioning attitude
1,The number of violations has increased.,leadership safety values and actions,work processes,continuous learning,NaN,NaN,NaN
2,The DCISC has identified a number of potential nuclear safety issues with the use of closed cooling at DCPP.,effective safety communication,NaN,NaN,NaN,NaN,NaN
3,New regulatory requirements were not adequately translated into a specific calculation in the plant’s design basis and therefore failed to demonstrate that the 230kV preferred offsite power source had adequate capacity and capability to supply the minimum required terminal voltage to plant engineering safety features following a limiting transmission system contingency.,leadership safety values and actions,work processes,NaN,NaN,NaN,NaN
4,"The DCISC learned in December 2013 that 16 impaired fire doors would not be repaired or replaced until 2017 due to funding deferrals and found this unacceptable. Following up in March 2014, the DCISC found that six doors had been repaired or replaced, and the remaining ten were the highest priority on the Plant Door Life Cycle Management Plan.",leadership safety values and actions,personal accountability,effective safety communication,NaN,NaN,NaN
5,"The DCPP Fuel Handling System has been problematic since Refueling Outage 2R14 (March 2009). Problems have been mostly due to age-related issues and lack of adequate inspection, maintenance, and component replacement, especially electrical contacts.",problem identification and resolution,NaN,NaN,NaN,NaN,NaN
6,"Additional efforts also need to be devoted to reducing operator burdens and workarounds as well the backlog of deficient critical components, and this requires the involvement of other station work groups besides Operations.",personal accountability,work processes,continuous learning,NaN,NaN,NaN
7,The loss of power to Unit 2 4kV Bus G during Refueling Outage 2R17 was avoidable and was due to a number of breakdowns in the planning and conduct of a maintenance activity during a refueling outage.,leadership safety values and actions,work processes,NaN,NaN,NaN,NaN
8,"Three Station Level Human Performance Event Clock Resets occurred during the fourth quarter of 2013, causing the station’s 18-month indicator for such Resets to become Yellow (deficient). Two of these three events involved Operations personnel. Operations performance with respect to human error rate has been Red (Unsatisfactory) since July 2013. Component mispositioning appears to be a contributor.",leadership safety values and actions,work processes,continuous learning,NaN,NaN,NaN
9,"Equipment problems and failures increased the frequency of Critical Event Clock Resets in the second half of 2013. Also, equipment problems due to aging have led to an increasingly negative trend in the station’s Deficient Critical Component Backlog Orders. DCPP’s performance on reducing or eliminating Safety System Functional Failures (SSFFs) has not improved despite implementation of a corrective action plan.",personal accountability,environment for raising concerns,effective safety communication,questioning attitude,NaN,NaN


In [ ]:
# predict similarity scores
model_docs_lbl_similarities = Lbl2Vec_model.predict_model_docs()